# Reach-Scale Modeling Analysis

This Notebook covers preliminary analysis for the design of a simple reach-scale model in TempEst-NEWT/NEXT.

The reach model aims to be as simple as possible while achieving two goals with reasonable accuracy:

- General goal: allow efficient estimation of reach temperature profiles, particularly downstream of observation points or disturbances such as reservoirs.  (Thus, it must be possible to provide arbitrary upstream data, not just a NEWT model.)
- Specific goal: account for key in-channel fluxes such as solar radiation (riparian shading), evaporation, and general atmospheric heat exchange, which cannot be accounted for by a lumped watershed model.  This allows model users to analyze key reach-scale phenomena.

It is anticipated that this can be achieved with a simple equilibrium model, where (for water temperature $T$ and air temperature $T_a$ in C, time $t$, and fitting coefficients $A,~B$):

$$
    \frac{dT}{dt} = A(T_a-T) + B
$$

The fitting coefficient $A$ lumps together a variety of influences on sensible heat flux, such as channel depth.  The fitting coefficient $B$ parametrizes a number of effects where their variation with stream temperature probably cannot be estimated for an ungaged watershed, and which are therefore assumed to be constant (with respect to temperature, not time).  These include, chiefly, solar radiation, evaporation, and lateral/groundwater inflows.

For model training purposes, we do not have densely-spaced observations, and it is necessary to apply uniform coefficients.  Over long distances, applying the above equation linearly would result in substantial error, so we use an exponential solution:

$$
    T - T_a = C\exp(-At) + Bt \implies T = C\exp(-At) + Bt + T_a
$$

Such that:

- If $B$ is zero, $T$ exponentially decays towards $T_a$
- C = $T-T_a$ at the initial step
- A and B are fitted

A further necessity is to estimate the travel time over a reach, $t$.  With uniform flow, we can begin from Manning's equation, $v=\frac1n R^{2/3}S^{1/2}$ (for velocity $v$, roughness $n$, hydraulic radius $R$, and slope $S$).  Assuming - somewhat dubiously, but of necessity without coupling a hydrologic model - that $R$ has a power-law relationship to flow $Q$, which in turn has a power law relationship to the product of area $A$ and precipitation $P$, we have $v=\frac1{\alpha}\frac1n(AP)^{\beta}S^{1/2}$ for fitting coefficients $\alpha,~\beta$.  Thus, for reach length $L$ and fitting coefficient 

$$
    t = \frac{\alpha Ln}{(AP)^{\beta}\sqrt{S}}
$$

Now, for the CONUS, we can get $n$, $L$, $S$ from NHDPlus Value Added Attributes, and only need to estimate $\alpha,~\beta$.

Note that for fitting a single river, those are all constant, so we (introducing a few more fitting coefficients, because we don't have enough to keep track of) simply have $t=\frac{k}{AP^q}$, and therefore need to fit $A,~B,~C,~k,~q$.  Fitting five coefficients is pretty easy with a reasonably long timeseries, and since we're using a simple analytic solution, we can very quickly calculate tons of permutations and identify a rock-solid optimum.  If our assumptions are sufficiently decent, of course.

Now, how do we get the training data?  There are two possible approaches there.